In [ ]:
!git clone https://github.com/PhamMaiLinh-2403/VN_tourism

Cloning into 'VN_tourism'...
remote: Enumerating objects: 389, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 389 (delta 4), reused 7 (delta 2), pack-reused 376 (from 1)
Receiving objects: 100% (389/389), 44.66 MiB | 25.58 MiB/s, done.
Resolving deltas: 100% (241/241), done.


In [ ]:
cd VN_tourism

/content/VN_tourism


In [ ]:
import os
os.listdir()

['main.ipynb',
 '.git',
 'data',
 'README.md',
 '.gitignore',
 'Notes về cách chunk.txt']

In [ ]:
pip install chromadb # sentence_transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 15.3 MB/s

# Import libraries

In [ ]:
import glob
import os
from sentence_transformers import SentenceTransformer
from chromadb import PersistentClient
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import uuid
import shutil


# Config

In [ ]:
BASE_MODEL = 'keepitreal/vietnamese-sbert'
# BASE_MODEL = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
COLLECTION_NAME = 'VN_tourism'
DB_DIR = 'data/chroma_db'
TOP_K = 10
regions = {
    'Hà Giang': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Cao Bằng': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Lạng Sơn': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Bắc Kạn': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Thái Nguyên': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Tuyên Quang': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Quảng Ninh': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Lào Cai': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Yên Bái': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Phú Thọ': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Bắc Giang': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Đông Bắc Bộ)',
    'Lai Châu': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Điện Biên': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Sơn La': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Hòa Bình': 'Miền Bắc - Trung du và miền núi Bắc Bộ (Tây Bắc Bộ)',
    'Hà Nội': 'Miền Bắc - Đồng bằng sông Hồng',
    'Hải Phòng': 'Miền Bắc - Đồng bằng sông Hồng',
    'Vĩnh Phúc': 'Miền Bắc - Đồng bằng sông Hồng',
    'Bắc Ninh': 'Miền Bắc - Đồng bằng sông Hồng',
    'Hải Dương': 'Miền Bắc - Đồng bằng sông Hồng',
    'Hưng Yên': 'Miền Bắc - Đồng bằng sông Hồng',
    'Hà Nam': 'Miền Bắc - Đồng bằng sông Hồng',
    'Nam Định': 'Miền Bắc - Đồng bằng sông Hồng',
    'Thái Bình': 'Miền Bắc - Đồng bằng sông Hồng',
    'Ninh Bình': 'Miền Bắc - Đồng bằng sông Hồng',
    'Thanh Hóa': 'Miền Trung - Bắc Trung Bộ',
    'Nghệ An': 'Miền Trung - Bắc Trung Bộ',
    'Hà Tĩnh': 'Miền Trung - Bắc Trung Bộ',
    'Quảng Bình': 'Miền Trung - Bắc Trung Bộ',
    'Quảng Trị': 'Miền Trung - Bắc Trung Bộ',
    'Thừa Thiên Huế': 'Miền Trung - Bắc Trung Bộ',
    'Đà Nẵng': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Quảng Nam': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Quảng Ngãi': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Bình Định': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Phú Yên': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Khánh Hòa': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Ninh Thuận': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Bình Thuận': 'Miền Trung - Duyên hải Nam Trung Bộ',
    'Kon Tum': 'Miền Trung - Tây Nguyên',
    'Gia Lai': 'Miền Trung - Tây Nguyên',
    'Đắk Lắk': 'Miền Trung - Tây Nguyên',
    'Đắk Nông': 'Miền Trung - Tây Nguyên',
    'Lâm Đồng': 'Miền Trung - Tây Nguyên',
    'Thành phố Hồ Chí Minh': 'Miền Nam - Đông Nam Bộ',
    'Bà Rịa - Vũng Tàu': 'Miền Nam - Đông Nam Bộ',
    'Bình Dương': 'Miền Nam - Đông Nam Bộ',
    'Bình Phước': 'Miền Nam - Đông Nam Bộ',
    'Đồng Nai': 'Miền Nam - Đông Nam Bộ',
    'Tây Ninh': 'Miền Nam - Đông Nam Bộ',
    'Cần Thơ': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Long An': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Tiền Giang': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Bến Tre': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Vĩnh Long': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Trà Vinh': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Hậu Giang': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Sóc Trăng': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Đồng Tháp': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'An Giang': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Kiên Giang': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Bạc Liêu': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)',
    'Cà Mau': 'Miền Nam - Đồng bằng sông Cửu Long (Tây Nam Bộ)'
}

# Create database

In [ ]:
def extract_headings(data_list):
    heading = []
    for head in data_list:
        head = head.strip('\n').strip().split('\n')[0]
        if len(head) > 0:
            heading.append(head)
        else:
            continue
    return heading

def extract_chunks_from_file(data_path):
    '''
    Extract thành các chunks cho từng tỉnh
    '''
    # Đọc data
    with open(data_path, 'r', encoding='utf-8') as f:
        data = f.read()

    # Trích xuất tên vùng
    province = os.path.splitext(os.path.basename(data_path))[0]
    region = regions[province]
    area = region + ' - ' + province

    # Tách các phần của data ra
    data_heading_split = data.split('\n\n\n')
    data_heading_split = [i for i in data_heading_split if len(i) > 0]

    # Trích xuất heading của các data (heading cho các phần lớn, bắt buộc phải có)
    # heading = []
    # for head in data_heading_split:
    #     head = head.strip('\n').split('\n')[0]
    #     if len(head) > 0:
    #         heading.append(head)
    #     else:
    #         continue
    heading = extract_headings(data_heading_split)
    chunked_data = []

    # Chunk thành các data
    for i in range(len(data_heading_split)):
        data_heading_split[i] = data_heading_split[i].replace(f'{heading[i]}\n', '')
        # Trong trường hợp phần quá dài
        if len(data_heading_split[i].split()) > 250:
            # Split theo \n\n
            data_heading_split[i] = data_heading_split[i].strip().strip('\n').split('\n\n')

            # Xét các chunks con trong phần lớn
            for j in range(len(data_heading_split[i])):
                # Chunks con quá dài
                if len(data_heading_split[i][j].split()) > 250:
                    # Tách possible heading ra
                    possible_heading = data_heading_split[i][j].split('\n')[0]
                    if len(possible_heading) > 10:
                        possible_heading = ''

                    # Tách chunks theo \s\s\n
                    small_data = data_heading_split[i][j].strip().strip('\n').split('  \n')
                    for sd in small_data:
                        # Nếu chunk đã tách theo \s\s\n vẫn có độ dài quá dài thì chunks theo số lượng từ
                        if len(sd.split()) > 250:
                            sd = sd.split()
                            for k in range(0, len(sd), 250):
                                if k + 250 > len(sd):
                                    end = len(sd)
                                else:
                                    end = k + 250
                                paragraph = ' '.join(sd[k:end])
                                if len(possible_heading) > 0:
                                    chunked_data.append((area + '. ' + heading[i] + '. ' + possible_heading + '. ' + paragraph).replace('\n', ' '))
                                else:
                                    chunked_data.append((area + '. ' + heading[i] + '. ' + paragraph).replace('\n', ' '))

                else:
                    # chunked_data.append(province + '\n' + heading[i] + '\n' + data_heading_split[i][j])
                    possible_heading = data_heading_split[i][j].split('\n')[0]
                    if len(possible_heading) > 10:
                        chunked_data.append((area + '. ' + heading[i] + '. ' + data_heading_split[i][j]).replace('\n', ' '))
                    else:
                        chunked_data.append((area + '. ' + heading[i] + '. ' + possible_heading + '. ' + data_heading_split[i][j]).replace('\n', ' '))

            # chunked_data.append(province + '\n' + data_heading_split[i][0])
            # for j in range(1, len(data_heading_split[i])):
            #     data_heading_split[i][j] = province + '\n' + heading[i] + '\n' + data_heading_split[i][j]
            #     chunked_data.append(data_heading_split[i][j])
        else:
            chunked_data.append((area + '. ' + heading[i] + '. ' + data_heading_split[i]).replace('\n', ' '))

    return chunked_data

def extract_chunks(path_to_folder, general_file_path):
    '''
    Pipeline extract chunks cho toàn bộ file data trong folder
    '''
    # Trích xuất path tới các file
    folder = path_to_folder
    files = glob.glob(f'{folder}/*.txt')

    with open(general_file_path, 'r', encoding='utf-8') as f:
        general_file = f.read()

    # Lấy thông tin từ trong file general trước
    chunks = general_file.split('\n\n')

    # Trích xuất chunks từ các file vùng miền cá nhân
    for file in files:
        chunks += extract_chunks_from_file(file)

    return chunks

def create_collection(client, name):
    existing = [c.name for c in client.list_collections()]
    if name in existing:
        print(f"⚠️ Collection '{name}' exists → deleting…")
        client.delete_collection(name)

        # # Optional: also delete the underlying persisted folder
        # if os.path.exists(db_dir):
        #     shutil.rmtree(db_dir)
        #     print("🗑️ Persist directory removed to avoid leftover data.")

    # If you want Chroma to embed automatically:
    embedder = SentenceTransformerEmbeddingFunction(model_name=BASE_MODEL)

    print("Creating fresh collection…")
    return client.create_collection(
        name=name,
        embedding_function=embedder,
        metadata={"hnsw:space": "cosine"}
    )

    # print("Creating fresh collection…")
    # return client.create_collection(name=name)

def embed_and_store_chunks(chunks, model, collection):
    print(f"Embedding {len(chunks)} chunks...")

    embeddings = model.encode(chunks, convert_to_numpy=True, normalize_embeddings=True)
    ids = [str(uuid.uuid4()) for _ in chunks]

    collection.add(
        documents=chunks,
        embeddings=embeddings.tolist(),
        ids=ids
    )

    print("✓ Stored in ChromaDB!")

def create_database(path_to_folder, general_file_path):
    # Create chunks
    chunks = extract_chunks(path_to_folder, general_file_path)
    print(f'Total chunks extracted: {len(chunks)}')

    #Initialize model + db
    print('Initializing model and database...')
    model = SentenceTransformer(BASE_MODEL)
    model.tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
    chroma_client = PersistentClient(path=DB_DIR)

    # Create DB collection
    print('Creating chromadb collection...')
    collection = create_collection(chroma_client, COLLECTION_NAME)

    # Add data embeddings
    print('Start embedding data...')
    embed_and_store_chunks(chunks, model, collection)
    print("✓ Database saved.")

path_to_folder = 'data/texts'
general_info = 'data/vietnam_tourism.txt'
# chunks_new = extract_chunks(path_to_folder, general_info)
create_database(path_to_folder, general_info)

Total chunks extracted: 2136
Initializing model and database...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating chromadb collection...
Creating fresh collection…
Start embedding data...
Embedding 2136 chunks...
✓ Stored in ChromaDB!
✓ Database saved.


In [ ]:
from google.colab import files
import shutil

shutil.make_archive("chromadb", 'zip', '/content/VN_tourism/data/chroma_db')
files.download('chromadb.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Retrieve context

In [ ]:
embedding_model = SentenceTransformer(BASE_MODEL)
chroma_client = PersistentClient(path=DB_DIR)
# if COLLECTION_NAME not in [c.name for c in chroma_client.list_collections()]:
#     create_database(path_to_folder, general_info)
collection = chroma_client.get_collection(COLLECTION_NAME)

# def extract_context(query, k = TOP_K):
#     # Embed query
#     query_embedding = embedding_model.encode(query, convert_to_numpy=True, normalize_embeddings=True)
#     # if COLLECTION_NAME not in [c.name for c in chroma_client.list_collections()]:
#     #     create_database(path_to_folder, general_info)
#     # collection = chroma_client.get_collection(COLLECTION_NAME)

#     results = collection.query(
#         query_embeddings=query_embedding,
#         n_results=k,
#         include=['documents', 'distances']
#     )

#     retrieved_docs = results['documents'][0]
#     distances = results['distances'][0]
#     return retrieved_docs, distances

# query = 'Thanh Hóa nên tham quan điểm nào?'
# docs, distances = extract_context(query)

# Build Prompt

In [ ]:
def build_prompt(query, retrieved_docs):
    context = '\n---------\n'.join(retrieved_docs)
    prompt = f"""
Bạn là một chuyên gia hàng đầu về du lịch Việt Nam. Hãy sử dụng các thông tin được cho để trả lời câu hỏi. Kiểm tra các tài liệu trước khi cho vào, nhất định phải có từ khóa trong câu hỏi.
Tuyệt đối không dựa vào kiến thức cá nhân hay bên ngoài. Chỉ được trả lời bằng tiếng Việt. Nếu văn bản không phải tiếng Việt, dịch ra trước rồi chuyển.

Câu hỏi:
{query}

Ngữ cảnh:
{context}
"""
    return prompt

# prompt = build_prompt(query, docs)
# print(prompt)

# Run UI

In [ ]:
# BASE_MODEL = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
# DB_DIR = 'data/chroma_db'
# COLLECTION_NAME = 'VN_tourism'

# Determine device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Initialize embedding model & database (đã gọi ở trên)
# embedding_model = SentenceTransformer(BASE_MODEL)
# chroma_client = PersistentClient(path=DB_DIR)
# collection = chroma_client.get_collection(COLLECTION_NAME)

# Load local LLM - SỬ DỤNG FLOAT32 THAY VÌ FLOAT16
def load_model(model_name): # Cho Qwen/Qwen2.5-1.5B-Instruct
  tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      dtype=torch.float32,  # Dùng float32 thay vì float16
      low_cpu_mem_usage=True
  ).to(device)  # Tải lên GPU nếu có, ngược lại CPU

  return model, tokenizer
model, tokenizer = load_model('Qwen/Qwen2.5-1.5B-Instruct')

def load_phogpt_model(model_name="vinai/PhoGPT-4B-Chat"):
  config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
  config.init_device = device
  tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
  model = AutoModelForCausalLM.from_pretrained(model_name, config=config, torch_dtype=torch.bfloat16, trust_remote_code=True)

  return model, tokenizer

# model,tokenizer = load_phogpt_model()

def extract_context(query, k=TOP_K):
    query_embedding = embedding_model.encode(query, convert_to_numpy=True, normalize_embeddings=True)
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=k,
        include=['documents']
    )
    return results['documents'][0]

# def build_prompt(query, retrieved_docs):
#     context = '\n'.join(retrieved_docs[:3])
#     prompt = f"""Bạn là chuyên gia du lịch Việt Nam. Trả lời dựa vào thông tin sau:

# Câu hỏi: {query}

# Thông tin:
# {context}

# Trả lời:"""
#     return prompt

def chat_bot(message, history):
    try:
        docs = extract_context(message, k=TOP_K)
        prompt = build_prompt(message, docs)

        messages = [{"role": "user", "content": prompt}]
        text = tokenizer.apply_chat_template(messages, tokenize=False)
        inputs = tokenizer(text, return_tensors="pt").to(device) # Chuyển inputs sang GPU nếu có

        with torch.no_grad():  # Tiết kiệm memory
            outputs = model.generate(
                **inputs,
                max_new_tokens=10000,
                temperature=0.7,
                top_p=0.9,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode only the newly generated tokens, excluding the prompt
        answer = tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
        return answer
    except Exception as e:
        return f"❌ Lỗi: {str(e)}"

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Câu hỏi về du lịch Việt Nam", lines=2)
    submit_btn = gr.Button("Gửi")

    def respond(m, hist):
        bot = chat_bot(m, hist)
        hist.append((m, bot))
        return "", hist

    submit_btn.click(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

Using device: cuda


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

/tmp/ipython-input-3343002503.py:83: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
/tmp/ipython-input-3343002503.py:83: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://077afe83d2ebeca277.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# keepitreal/vietnamese-sbert
extract_context('Thanh Hóa có những điểm tham quan nào?', k=10)

from sklearn.metrics.pairwise import cosine_similarity

def rerank_by_embedding(query_emb, docs_emb, docs):
    scores = cosine_similarity([query_emb], docs_emb)[0]
    ranked_docs = [doc for _, doc in sorted(zip(scores, docs), reverse=True)]
    return ranked_docs

rerank_by_embedding()

['Miền Trung - Bắc Trung Bộ - Thanh Hóa. Chơi đâu. Thanh Hóa có nguồn tài nguyên du lịch phong phú với nhiều danh lam thắng cảnh, di tích lịch sử, bãi biển, rừng nguyên sinh, hệ thống hang động và núi đá vôi trải rộng khắp các huyện của tỉnh cùng nhiều lễ hội nổi tiếng. Địa điểm du lịch nổi tiếng xứ Thanh là bãi biển Sầm Sơn. Hàng năm, bãi biển thu hút một lượng lớn khách du lịch, đặc biệt là vào dịp nghỉ lễ 30/4 và mùa hè. Ngoài ra, Thanh Hóa còn nhiều điểm đến nổi tiếng khác.',
 'Miền Trung - Bắc Trung Bộ - Quảng Bình. Điểm tham quan và vui chơi. Sông Chày - Hang Tối Sông Chày bắt nguồn từ khối núi đá vôi chảy quanh qua các làng, kết hợp với hang Tối, hang có hệ thống thạch nhũ cổ tiêu biểu cho quá trình kiến tạo địa chất của khu vực Phong Nha - Kẻ Bàng tạo thành một tuyến du lịch khám phá thiên nhiên. Hiện Sông Chày - Hang Tối đang được khai thác theo loại hình thám hiểm với nhiều dịch vụ như zipline tắm sông, zipline khám phá hang động, chèo thuyền kayak khám phá sông Chày, bơi qua

- Embedding Models:
  - multi j j đấy: dở như hạch
  - keepitreal/vietnamese-sbert: khá hơn, retrieve 2 kết quả đầu đã chính xác nhưng còn miss
  - dangvantuan/vietnamese-embedding: max token bé tí mãi vẫn không embed nổi
  - VoVanPhuc/sbert-vi-v2: không public
- LLM model:
  - Qwen2.5-1.5B-Instruct: Cũng được nhưng rối loạn ngôn ngữ nhiều
  - Qwen2.5-3B-Instruct:
  - ngoan/Llama-2-7b-vietnamese-20k: xài hết RAM luôn
  - ura-hcmut/ura-llama-2.1-8b
  - bkai-foundation-models/vietnamese-llama2-7b-120GB
- GPT đang rec rerank